In [1]:
# Importing necessary packages:
import re
import os
from glob import glob

import math
import numpy as np
import datetime
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines



import seaborn as sns

import scipy.integrate as integrate
import scipy.special as special

from sympy import elliptic_pi

from PipeLine import *

In [ ]:
bug_files = glob("../sumrule_data/N*-analyze_bug/N*.csv")
geometry = 'cylinder'
df_name = 'all_in_one-ens_avg'
ext=['-properties-ens_avg.csv']
ens_evg_properties_files = PipeLine.file_reader(bug_files,extensions=ext)
filename = 'properties-ens_avg-all_in_one'
ens_avg_properties_df = PipeLine.all_in_one_properties(ens_evg_properties_files,df_name,geometry,filename=filename,round_to=4)

In [ ]:
#vfrc_crowds = list(ens_avg_properties_df.vfrc_crowd.sort_values().drop_duplicates())
#vfrc_colors = sns.color_palette("coolwarm", len(vfrc_crowds))
dcrowds = list(ens_avg_properties_df.dcrowd.sort_values().drop_duplicates())
print(dcrowds)
dcyls = list(ens_avg_properties_df.dcyl.sort_values().drop_duplicates())
print(dcyls)
nmons = list(ens_avg_properties_df.nmon.sort_values().drop_duplicates())
vfrc_crowds_selected = [0.1,0.2,0.3,0.4]

line_styles = ['--',':','-.',(0, (3, 1, 1, 1, 1, 1))]
ls_cycle = cycler('ls', line_styles) # equal to len(dcrowds)

line_widths = np.arange(1,3,1)
lw_cycle = cycler('lw', line_widths) # equal to len(nmons

marker_names = [r'$\frac{\phi_c(r)}{a_c}$',r'$\frac{\phi_m(r)}{a}$',r'$\sum\frac{\phi_i(r)}{a_i}$']
marker_styles = 'os^'
marker_cycle = cycler('marker',marker_styles)

vfrc_colors_selected = sns.color_palette("Set2", len(vfrc_crowds_selected))#,as_cmap=True)
#vfrc_colors_selected = sns.color_palette("hls", len(vfrc_crowds_selected))#,as_cmap=True)
vfrc_colors_selected

In [ ]:
xcol="vfrc_crowd"
df=ens_avg_properties_df
fontsize=20
colors = sns.color_palette(n_colors=4, palette="Set2", desat=1)
leg_labels = [r'$\frac{D}{a}$ (Line color)', '15.0', '20.0', '25.0','30.0', r'$N$ (Line size)', 1000, 2000, r'$\frac{a_c}{a}$ (Marker type)', '1.0', '2.0', '4.0']
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd_eff", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_eff_normalized", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_normalized", leg_labels, colors, fontsize=20)

In [ ]:
def organize_hists_ens_evg(pair_csvs,properties,xname,yname,xnorm,ynorm,filename=None):
    """
    This funcions receives a list of paths to histogram data (each contains two columns; one bins, other counts) in different directions and return a dict of paris of filenames and Pandas dataframes as pairs of key and values.
    """
    group_dfs = []
    properties[['groupname','garbage']] = properties.filename.str.split(pat='-',expand=True)
    for crd_csv, mon_csv in pair_csvs:
        crd_df = pd.read_csv(crd_csv,index_col=0)
        mon_df = pd.read_csv(mon_csv,index_col=0)
        group_name = list(crd_df.columns)[0].split('-')[0]
        
        cell_attrs = PipeLine.cellAttributes(group_name,warning=False)
        x_normalizer = {'dcyl':cell_attrs.dcyl, 'lcyl':cell_attrs.lcyl, 'degree': 180}
        y_normalizer = {'hist':1, 'rho':{'crd':1,'mon': 1}, 'phi': {'crd':(2 *  cell_attrs.ncrowd * cell_attrs.dcrowd ** 3) / (3 * cell_attrs.dcyl ** 2 * cell_attrs.lcyl),'mon': (2 *  cell_attrs.nmon * cell_attrs.dmon ** 3) / (3 * cell_attrs.dcyl ** 2 * cell_attrs.lcyl)}}
    
        crd_df = pd.read_csv(crd_csv,names=[xname,yname+'_c'],skiprows=1)
        mon_df = pd.read_csv(mon_csv,names=[yname+'_m'],skiprows=1,usecols=[1])
        
        crd_df[xname+'_norm'] = 2 * crd_df[xname] / x_normalizer[xnorm]
        crd_df[yname+'_norm'+'_c'] = crd_df[yname+'_c'] / y_normalizer[ynorm]['crd']
        crd_df[yname+'_norm'+'_c'] = crd_df[yname+'_norm'+'_c'] / crd_df[yname+'_norm'+'_c'].sum()
        

        mon_df[yname+'_norm'+'_m'] = mon_df[yname+'_m'] / y_normalizer[ynorm]['mon']
        mon_df[yname+'_norm'+'_m'] = mon_df[yname+'_norm'+'_m'] / mon_df[yname+'_norm'+'_m'].sum()
        
        # other properties:
        
        merged_df = pd.concat([crd_df,mon_df],axis=1)
        merged_df[yname+'_sum'] = (merged_df[yname+'_c'] + merged_df[yname+'_m']) / (y_normalizer[ynorm]['crd'] + y_normalizer[ynorm]['mon'])
        merged_df[yname+'_norm_sum'] = merged_df[yname+'_sum'] / merged_df[yname+'_sum'].sum()
        for col in properties.columns:
            cond = properties['groupname'] == group_name
            merged_df[col] = properties[cond][col].values[0]
        merged_df.drop(['filename','garbage'],axis=1,inplace=True)
        group_dfs.append(merged_df)
    
    df_of_hists = pd.concat(group_dfs)
    df_of_hists.reset_index(inplace=True,drop=True)
    if filename != None:
        outname = filename+'.csv'
        df_of_hists.to_csv(outname)
        
    return df_of_hists

In [ ]:
properties_df = pd.read_csv('./properties-ens_avg-all_in_one.csv',header=0)
all_files = glob("../sumrule_data/*-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext = ['-rPhisCrd-ens_avg.csv','-rPhisMon-ens_avg.csv']
rPhis_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_df = PipeLine.organize_hists_ens_evg(rPhis_ens_avg_csvs,properties_df,xname,
                                                   yname,xnorm,ynorm,filename='rPhis-ens_avg-all_in_one')

In [ ]:
rPhis_df_sel_phis = rPhis_ens_avg_df[rPhis_ens_avg_df['vfrc_crowd'].isin([0.1,0.2,0.3,0.4])]

In [ ]:
rPhis_df_sel_phis.info()

In [ ]:
#fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
dcrowd_grid = sns.relplot(x="r_norm", y="phi_r_norm_sum",kind="line",hue='vfrc_crowd',palette="Set2",
            col='dcrowd',col_wrap=2,style='dcyl',data=rPhis_df_sel_phis,alpha=0.7)

In [ ]:
dcyl_grid = sns.relplot(x="r_norm", y="phi_r_norm_sum",kind="line",hue='vfrc_crowd',palette="Set2",
            col='dcyl',col_wrap=2,style='dcrowd',data=rPhis_df_sel_phis,alpha=0.7)

In [ ]:
phic_grid_dcrowd_color = sns.relplot(x="r_norm", y="phi_r_norm_sum",kind="line",hue='dcrowd',palette="Set2",
            col='vfrc_crowd',col_wrap=2,style='dcyl',data=rPhis_df_sel_phis,alpha=0.7)

In [ ]:
phic_grid_dcyl_color = sns.relplot(x="r_norm", y="phi_r_norm_sum",kind="line",hue='dcyl',palette="Set2",
            col='vfrc_crowd',col_wrap=2,style='dcrowd',data=rPhis_df_sel_phis,alpha=0.7)

In [ ]:
rPhis_df_sel_dcrowds1 = rPhis_ens_avg_df[(rPhis_ens_avg_df['vfrc_crowd'].isin([0.1,0.2,0.3,0.4])) & (rPhis_ens_avg_df['dcrowd']==2.0)]
rPhis_df_sel_dcrowds2 = rPhis_ens_avg_df[(rPhis_ens_avg_df['vfrc_crowd'].isin([0.1,0.2,0.3,0.4])) & (rPhis_ens_avg_df['dcrowd'] == 4.0) & (rPhis_ens_avg_df['dcyl'] == 30.0)]
rPhis_df_sel_dcrowds = pd.concat([rPhis_df_sel_dcrowds1,rPhis_df_sel_dcrowds2],axis=0)
rPhis_df_sel_dcrowds.reset_index(inplace=True,drop=True)
rPhis_df_sel_dcrowds.info()

In [ ]:
yaxes_grid = sns.FacetGrid(rPhis_df_sel_dcrowds, col='dcyl',col_wrap=2,sharey=False,height=6,legend_out=True)
yaxes_grid.map_dataframe(sns.lineplot, x="r_norm", y="phi_r_norm_c",hue='vfrc_crowd',lw=1,ls='--',marker='o',markersize=7,markevery=10,palette="Set2")
yaxes_grid.map_dataframe(sns.lineplot, x="r_norm", y="phi_r_norm_m",hue='vfrc_crowd',lw=1,ls=':',marker='s',markersize=7,markevery=10,palette="Set2")
yaxes_grid.map_dataframe(sns.lineplot, x="r_norm", y="phi_r_norm_sum",hue='vfrc_crowd',lw=1,ls='-.',marker='^',markersize=7,markevery=10,palette="Set2",legend='full')
yaxes_grid.tight_layout()
#yaxes_grid.map_dataframe(sns.lineplot, x="r_norm", y="phi_r_norm_sum")
#yaxes_grid.map_dataframe(sns.lineplot, x="r_norm", y="phi_r_norm_sum")
#yaxes_grid = sns.relplot(x="r_norm", y="phi_r_norm_sum",kind="line",hue='vfrc_crowd',palette="Set2",
 #           col='dcyl',col_wrap=2,style='dcrowd',marker='s',markersize=7,markevery=10,data=rPhis_df_sel_dcrowds,alpha=0.7)

## The sum rule in the radial direction:

### 1. $a_c$ as subplots

### 2. $D$ as sub-plots

### 3. $\phi_c$ as sub-plots

In [ ]:
def sumrule_line(vfrc_cs,dcrowds,direction):
    sumrule = [vfrc_c*dcrowd for vfrc_c, dcrowd in zip([vfrc_cs],[dcrowds])]
    sumrule = np.ones((len(direction),2)) * sumrule
    sumrule[:,0] = direction
    return sumrule
jafar = sumrule_line(0.3,2,rPhi_ens_avg['r_norm_c'].to_numpy())
fig, axis = plt.subplots(nrows=1,ncols=1,figsize=(16,12),sharey='row')
axis.plot(jafar[:,0],jafar[:,1])

In [ ]:
all_files = glob("../sumrule_data/*ac2.0-analyze_all/N*.csv") + glob("../sumrule_data/*ac4.0-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext=['-rPhisCrd-ens_avg.csv']
rPhisCrd_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
ext=['-rPhisMon-ens_avg.csv']
rPhisMon_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_dfs = organize_hists_ens_evg(rPhisCrd_ens_avg_csvs,rPhisMon_ens_avg_csvs,xname,yname,xnorm,ynorm)

dcyls = [25.0,20.0,15.0,30.0]
dcrowds = [2.0,2.0,2.0,4.0]

In [ ]:
xname='r_norm_c'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22

for idx, (dcrowd, dcyl, axis) in enumerate(zip(dcrowds,dcyls,axes)):
    axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl)+'$,{a_c}/{a}=$'+'${}$'.format(dcrowd)+r': $\kappa=$'+'${:.2f}$'.format(dcrowd/(dcyl-dcrowd)),fontsize=fontsize)
    axis2 = axis.twinx()
    axis2.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
    axis.grid(True,ls=':',lw=1)
    axis.tick_params(axis ='both',direction='in')

    if idx == 0:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
    elif idx == 1: 
        PipeLine.yticks(axis,(0.0,0.02,0.004,0.0008),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.02,0.004,0.0008),code=True,fontsize=fontsize-6)
    elif idx == 2: 
        PipeLine.yticks(axis,(0.0,0.03,0.005,0.001),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.03,0.005,0.001),code=True,fontsize=fontsize-6)  
    else:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6) 
    axis.set_ylabel(r'$\phi_i(r)$',fontsize=fontsize-3)
    axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
    PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)


    lines = []
    for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
        cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
        vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
        if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
            line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=10, alpha=1,ls=':')
            line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=10, alpha=1,ls='--')
            line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=10, alpha=1)
            lines += line

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=2,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none',markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-6,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-all-selected_crowd-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
all_files = glob("../sumrule_data/*D25.0ac*-analyze_all/N*.csv") + glob("../sumrule_data/*D30.0ac6.0-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext=['-rPhisCrd-ens_avg.csv']
rPhisCrd_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
ext=['-rPhisMon-ens_avg.csv']
rPhisMon_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_dfs = organize_hists_ens_evg(rPhisCrd_ens_avg_csvs,rPhisMon_ens_avg_csvs,xname,yname,xnorm,ynorm)

dcyls = [25.0,25.0,25.0,30.0]
dcrowds = [1.0,2.0,4.0,6.0]

In [ ]:
xname='r_norm_c'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22


for idx, (dcrowd, dcyl, axis) in enumerate(zip(dcrowds,dcyls,axes)):
    axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl)+'$,{a_c}/{a}=$'+'${}$'.format(dcrowd)+r': $\kappa=$'+'${:.2f}$'.format(dcrowd/(dcyl-dcrowd)),fontsize=fontsize)
    axis2 = axis.twinx()
    axis2.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
    axis.grid(True,ls=':',lw=1)
    axis.tick_params(axis ='both',direction='in')

    if idx == 0:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
    elif idx == 1: 
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
    elif idx == 2: 
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)  
    else:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6) 
    axis.set_ylabel(r'$\phi_i(r)$',fontsize=fontsize-3)
    axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
    PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)


    lines = []
    for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
        cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
        vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
        if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
            line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=10, alpha=1,ls=':')
            line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=10, alpha=1,ls='--')
            line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=10, alpha=1)
            lines += line

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=2,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none',markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-6,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-all-selected_dcyl-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)